In [1]:
import myfunctions as mf
from skimage.io import imshow
import napari 

The code with either 4D filtering or napari viewer can only be run on the cluster.

In [ ]:
# mf.segment4D(exp=mf.exp_list()[0], filtering4D=False)

In [5]:
exp = mf.exp_list()[0]
first_volume = mf.load_volume(exp=exp, time=160, isImage=True, OS='Windows')

In [6]:
viewer = napari.Viewer()
viewer.add_image(first_volume, name='first volume')

<Image layer 'first volume' at 0x1cc555134c0>